<a href="https://colab.research.google.com/github/Buzaabah/livelossplot/blob/master/NL2SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd original\ copy

In [0]:
#Important parckages

%matplotlib inline

import re,string
import numpy as np
import pickle
import spacy
from spacy.symbols import ORTH

en_tok = spacy.load('en')



In [11]:
#Reading required data

train_data = open('atis.uw.train.txt').readlines()
train_data[:5]

["list all the flights that arrive at general mitchell international from various cities ||| SELECT DISTINCT flight_1.flight_id FROM flight flight_1 , airport airport_1 , airport_service airport_service_1 , city city_1 WHERE flight_1.to_airport = airport_1.airport_code AND airport_1.airport_code = 'MKE' AND flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code\n",
 "give me the flights leaving denver august ninth coming back to boston ||| SELECT DISTINCT flight_1.flight_id FROM flight flight_1 , airport_service airport_service_1 , city city_1 , airport_service airport_service_2 , city city_2 , days days_1 , date_day date_day_1 WHERE flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code AND city_1.city_name = 'DENVER' AND ( flight_1.to_airport = airport_service_2.airport_code AND airport_service_2.city_code = city_2.city_code AND city_2.city_name = 'BOSTON' AND flight_1.flight_days = 

In [12]:
test_data = open('atis.uw.test.txt').readlines()
test_data[:5]

["what flights go from dallas to phoenix ||| SELECT DISTINCT flight.flight_id FROM flight WHERE ( flight.from_airport IN ( SELECT airport_service.airport_code FROM airport_service WHERE airport_service.city_code IN ( SELECT city.city_code FROM city WHERE city.city_name = 'DALLAS' )) AND flight.to_airport IN ( SELECT airport_service.airport_code FROM airport_service WHERE airport_service.city_code IN ( SELECT city.city_code FROM city WHERE city.city_name = 'PHOENIX' )) )\n",
 "what flights go from phoenix to salt lake city ||| SELECT DISTINCT flight.flight_id FROM flight WHERE ( flight.from_airport IN ( SELECT airport_service.airport_code FROM airport_service WHERE airport_service.city_code IN ( SELECT city.city_code FROM city WHERE city.city_name = 'PHOENIX' )) AND flight.to_airport IN ( SELECT airport_service.airport_code FROM airport_service WHERE airport_service.city_code IN ( SELECT city.city_code FROM city WHERE city.city_name = 'SALT LAKE CITY' )) )\n",
 "i need an early flight f

In [0]:
# making a corpus

def make_corpus(corpus_path):
    en,sql = [],[]
    for line in train_data:
        if '\n' in line:
            new_line = line.replace('\n','')
            new_line = new_line.split('|||')
            en.append(new_line[0])
            sql.append(new_line[1])
    return en,sql

In [16]:
#unpacking the corpus

en, sql = make_corpus('atis.uw.train.txt')
en[:2],sql[:2]

(['list all the flights that arrive at general mitchell international from various cities ',
  'give me the flights leaving denver august ninth coming back to boston '],
 [" SELECT DISTINCT flight_1.flight_id FROM flight flight_1 , airport airport_1 , airport_service airport_service_1 , city city_1 WHERE flight_1.to_airport = airport_1.airport_code AND airport_1.airport_code = 'MKE' AND flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code",
  " SELECT DISTINCT flight_1.flight_id FROM flight flight_1 , airport_service airport_service_1 , city city_1 , airport_service airport_service_2 , city city_2 , days days_1 , date_day date_day_1 WHERE flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code AND city_1.city_name = 'DENVER' AND ( flight_1.to_airport = airport_service_2.airport_code AND airport_service_2.city_code = city_2.city_code AND city_2.city_name = 'BOSTON' AND flight_1.flight_

In [0]:
#pre-processing for tokenizing 
# Natural language tokens

def en_tokenizer(text):
    text = text.lower()
    return [t.text for t in en_tok.tokenizer(text)]
  
#SQL tokens
def en_tokenizer(text):
    text = text
    return [t.text for t in en_tok.tokenizer(text)]

In [18]:
# Tokens

en_toks = [en_tokenizer(text) for text in en]
sql_toks = [en_tokenizer(text) for text in sql]
en_toks[:2], sql_toks[:2]

([['list',
   'all',
   'the',
   'flights',
   'that',
   'arrive',
   'at',
   'general',
   'mitchell',
   'international',
   'from',
   'various',
   'cities'],
  ['give',
   'me',
   'the',
   'flights',
   'leaving',
   'denver',
   'august',
   'ninth',
   'coming',
   'back',
   'to',
   'boston']],
 [[' ',
   'SELECT',
   'DISTINCT',
   'flight_1.flight_id',
   'FROM',
   'flight',
   'flight_1',
   ',',
   'airport',
   'airport_1',
   ',',
   'airport_service',
   'airport_service_1',
   ',',
   'city',
   'city_1',
   'WHERE',
   'flight_1.to_airport',
   '=',
   'airport_1.airport_code',
   'AND',
   'airport_1.airport_code',
   '=',
   "'",
   'MKE',
   "'",
   'AND',
   'flight_1.from_airport',
   '=',
   'airport_service_1.airport_code',
   'AND',
   'airport_service_1.city_code',
   '=',
   'city_1.city_code'],
  [' ',
   'SELECT',
   'DISTINCT',
   'flight_1.flight_id',
   'FROM',
   'flight',
   'flight_1',
   ',',
   'airport_service',
   'airport_service_1',
   ',